In [1]:
import psycopg
import pandas as pd
from envs import  con_string
pd.set_option('display.max_colwidth', None)


In [2]:

class PostgesAPI:
    def __init__(self, con_string) -> None:
        self.con_string = con_string

    
    def print_msg(self, msg):
        _mes_len = 50
        _char = "-"
        _half = ((_mes_len - len(msg) -2 ) // 2) * _char
        print(_mes_len * _char)
        print(f"{_half} {msg} {_half}")
        print("")
        
    def insert(self, table_name, records):
        self.print_msg('insert')
        try:
            with psycopg.connect(self.con_string) as conn:
                with conn.cursor() as cur:
                    if isinstance(records, dict): records = [records] 
                    cols = ', '.join(list(records[0].keys()))
                    vals = ', '.join([f"{tuple(record.values())}" for record in records])
                    insert_statements = f"""INSERT INTO {table_name} ({cols}) VALUES {vals}"""
                    cur.execute(insert_statements)
                    conn.commit()
            return f'{len(records)} records were inserted'
        except Exception as e:
            print(e)
        

    def update(self, sql_statements):
        self.print_msg('update')
        try:
            with psycopg.connect(self.con_string) as conn:
                with conn.cursor() as cur:
                    cur.execute(sql_statements)
                    conn.commit()
            return 'SQL Executed successfully'
        except Exception as e:
            print(e)
        


    def select(self, sql_statements):
        self.print_msg('select')
        try:
            with psycopg.connect(self.con_string, row_factory = psycopg.rows.dict_row) as conn:
                with conn.cursor() as cur:
                    cur.execute(sql_statements)
                    # columns = tuple([desc[0] for desc in cur.description])
                    data = cur.fetchall()
            return  data
        except Exception as e:
            print(e)

    def create(self, sql_file):
        self.print_msg('create')
        try:
            with psycopg.connect(self.con_string) as conn:
                with conn.cursor() as cur:
                    with open(sql_file, 'r') as sql_file:
                        sql_stms = sql_file.read().split(';')
                        for stm in sql_stms:
                            print(stm)
                            cur.execute(stm)
                            conn.commit()
            return 'SQL Executed successfully'
        except Exception as e:
            print(e)


            
    # def delete(self, table_name, records):
    #     pass
    # def update(self, table_name, records):
    #     pass
    
        
        
sql_api = PostgesAPI(con_string)  

## **CRUD Operations**

#### **Create** the database from a SQL file

In [58]:
sql_api.create('test.sql')

--------------------------------------------------
--------------------- create ---------------------


DROP TABLE IF EXISTS test

CREATE TABLE test (
    id serial PRIMARY KEY,
    num integer,
    data text
)

INSERT INTO test (num, data) 
VALUES (6000, 'HHH'), (800, 'BBB')




'SQL Executed successfully'

#### **Select** 

In [28]:
sql_api.select('select * from test')

--------------------------------------------------
--------------------- select ---------------------



[{'id': 1, 'num': 6000, 'data': 'HHH'}, {'id': 2, 'num': 800, 'data': 'BBB'}]

#### **Insert One Or Many** 
The insert method takes arguments\
**TABLE NAME:** target table name\
**RECORDS:** a record dict or a list of records  

In [15]:
records = [
    {
        "num":9999,
        "data" : 'AAA'
    },
    {
       "num":800,
        "data" : 'BBB'
    }
]

table_name =  "test"


In [19]:
sql_api.insert(table_name, records[0])

--------------------------------------------------
--------------------- insert ---------------------



'1 records were inserted'

#### **Insert records from a csv into a table** 
To keep the API simple we manually create the DDL\
as an example lets create the ddl for the below CSV

In [8]:
df = pd.read_csv('medecin_sample.csv', index_col=False)
df.head()

,nom,genre,specialite,address_dsp,daira,telephone,lat,lng
0,DJELLAL ABDENACER,Homme,Cardiologie,Lotissement Thala ex Stade 1er Novembre,Tizi Ouzou,026215386,36.770019,3.050049
1,SAHEB Belkacem,Homme,Cardiologie,"lot THALA, Bd KRIM Belkacem, local 6, NV",Tizi Ouzou,026205113,36.703037,4.054835
2,AIT BELKACEM Djallil,Homme,Cardiologie,"cité b,NV, 1er étage, local 01",Tizi Ouzou,0666265781,36.702298,4.045919
3,REDDAD IDIR,Homme,Cardiologie,"Cité 600 logts Bt N, N° 559, NV",Tizi Ouzou,026217470,36.770000,3.049360
4,MELLAH SAMIA,Femme,Dermatologie,Cité les genets Bt F N° 01,Tizi Ouzou,0666494120,36.705320,4.060288


#### **GENERATE DDL**
Save the DDL into a sql file 

In [9]:
print(pd.io.sql.get_schema(df, 'doctors'))

CREATE TABLE "doctors" (
"nom" TEXT,
  "genre" TEXT,
  "specialite" TEXT,
  "address_dsp" TEXT,
  "daira" TEXT,
  "telephone" TEXT,
  "lat" REAL,
  "lng" REAL
)


#### **Create** the database from a SQL file

In [10]:
sql_api.create('baylek_schema.sql')

--------------------------------------------------
--------------------- create ---------------------

DROP TABLE IF EXISTS doctors

CREATE TABLE IF NOT EXISTS doctors (
    id serial PRIMARY KEY,
    nom  VARCHAR(100),
    genre VARCHAR(10),
    specialite VARCHAR(50),
    address_dsp VARCHAR(200),
    daira VARCHAR(50),
    telephone VARCHAR(15),
    lat DOUBLE PRECISION ,
    lng DOUBLE PRECISION
)


'SQL Executed successfully'

In [11]:
records = df.to_dict('records')

In [12]:
sql_api.insert('doctors', records)

--------------------------------------------------
--------------------- insert ---------------------



'35 records were inserted'